# Анализ поведения пользователей мобильного приложения

**Цель проекта:** Необходимо проанализировать поведение покупателей на основании логов пользователей и результатов А/А/В - эксперимента (изменение шрифта во всем приложении).

**Входные данные**: логи пользователей мабильного приложения по продаже продуктов питания.

**Ход исследования:**

Исследование пройдёт в четыре этапа:

* Обзор и предобработка данных;
* Анализ данных. Воронка событий;
* Анализ результатов А/А/В-эксперимента;
* Выводы.

## Обзор и предобработка данных

In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from plotly import graph_objects as go
from scipy import stats as st
import math as mth
import warnings; warnings.filterwarnings(action = 'ignore')

ModuleNotFoundError: No module named 'plotly'

In [ ]:
# чтение файлов с данными и сохранение в df

try:
    data = pd.read_csv('/datasets/logs_exp.csv', sep='\t')
except:
    data = pd.read_csv('logs_exp.csv', sep='\t')

In [ ]:
# ознакомимся с содержанием, типом данных и выведем первые 5 строк датафрейма
data.info()
data.head()

In [ ]:
# обзор содержания столбцов EventName и ExpId

display(data['EventName'].unique())
display(data['ExpId'].unique())

In [ ]:
# проверка на явные дубликаты и пропуски

display(data.duplicated().sum())
display(data.isna().sum())

**Предварительно:** мы выявили, что в датафрейме отсутствуют пропуски, но есть дубли. Самих событий всего 5. Что следует сделать:
* привести название столбцов к хорошему стилю (и для удобства)
* заменить типа данных в столбце EventTimestamp (похоже, что дата и время указаны в секундах)
* вынесьти дату в отдельную колонку для дальнейшего анализа
* проверить пересечение пользователей в группе

In [ ]:
# переименуем столбцы:

data = data.rename(columns={'EventName': 'event', 'DeviceIDHash': 'user_id', 
                            'EventTimestamp': 'date_time', 'ExpId': 'group_id'})

# заменим тип данных `datetime`: 

data['date_time'] = pd.to_datetime(data['date_time'], unit='s')

# добавим поле с датой:

data['date'] = pd.to_datetime(data['date_time'].dt.date)

# удалим дубликаты

data = data.drop_duplicates().reset_index(drop=True)

display(data.head(1))
data.info()

In [ ]:
# проверим пользователей на пересечение в группах.

data.groupby('user_id')['group_id'].nunique().reset_index().query('group_id > 1')

**Итог предобработки:**
* привели название столбцов к хорошему стилю
* заменили типа данных в столбце EventTimestamp на datetime64
* вынесли дату в отдельную колонку для дальнейшего анализа
* пользователей, которые участвуют одновременно в двух или более группах - не выявлено

## Анализ данных. Воронка событий

### Количество событий и пользователей в логе

In [ ]:
# посмотрим сколько всего событий и пользователей в логе
print('Всего событий:', data.shape[0],'Виды событий:', data['event'].nunique(),'Всего пользователей:',\
      data['user_id'].nunique())

In [ ]:
# посмотрим сколько в среднем событий приходится на пользователя
print('В среднем, событий на пользователя: {:.0f}'.format(data.shape[0] / data['user_id'].nunique()))

In [ ]:
# посмотрим сколько минимальное и максимальное кол-во событий приходится на пользователя:
events_by_user = data.groupby('user_id').agg({'event':'count'}).reset_index()

display(events_by_user['event'].describe())

В среднем, на пользователя приходится порядка 32-х событий. При этои, минимальное количество - всего одно, а максимальное - 2307 шт. Если говорить о медиане - это 20 событий на пользователя.

### За какой период данные: максимальная и минимальная дата

In [ ]:
# найдем максимальную и минимальную дату
print('Минимальная дата:', data['date_time'].min(), 'Максимальная дата:', data['date_time'].max())

In [ ]:
# гистограммы по дате и времени:

plt.title('Распределение по дате')
data['date_time'].hist(bins=7*24, figsize=(15, 5), alpha=0.5)
plt.ylabel('Количество событий')
plt.xlabel('Дата')
plt.xticks(rotation=20)
plt.show()

plt.title('Распределение по времени суток')
data['date_time'].dt.hour.hist(bins=24, figsize=(15, 5), alpha=0.5)
plt.ylabel('Количество событий')
plt.xlabel('Время суток')
plt.xticks(range(0, 23))
plt.show()

На первой гистограмме видно, что данные за июль - неполные. Технически в логи новых дней по некоторым пользователям могут «доезжать» события из прошлого  — это может «перекашивать данные».
Поэтому, чтобы избежать данной проблемы, берем данные с 01-08-2019 по 07-08-2019. Так же, мы видим, что основная активность пользователей происходит в дневное время.

Отбросим старые данные и посмотрим, сколько событий и пользователей мы потеряли.

In [ ]:
before_logs = data.shape[0]
before_users = data['user_id'].nunique()

print('До корректировки периода всего событий:', before_logs, 'всего пользователей:', before_users)

# удаляем данные за июль
data = data[data['date_time'] > '2019-08-01']

print('После корректировки периода всего событий:', data.shape[0],', всего пользователей:',data['user_id'].nunique())

print('Изменение количества событий:', 
      data.shape[0]- before_logs, 
      '({:.1%})'.format((data.shape[0]-before_logs)/before_logs))
print('Изменение количества пользователей:', 
      data['user_id'].nunique()- before_users, 
      '({:.1%})'.format((data['user_id'].nunique()-before_users)/before_users))

После удаления части пользователей и событий посмотрим на распределения в тестовых группах (по количеству уникальных пользователей и по количеству событий):

In [ ]:
# распределение пользователей по группам.
data_group = data.groupby('group_id').agg({'user_id': 'nunique'}).reset_index()
data_group['ratio'] = round(data_group['user_id']/data_group['user_id'].sum()*100)
data_group

In [ ]:
# распределение событий по группам
data.groupby('group_id').agg({'event': 'count'}).reset_index()

Количество пользователей в группах в целом сопоставимо: 248-й
группе пользователей больше на 1% чем в группах 246 и 247. Количество событий в группах немного разнится, но это не критично.

### Воронка событий

Посмотрим, какие события есть в логах, как часто они встречаются.

In [ ]:
events = (data.
          groupby('event').
          agg({'user_id': 'count'}).
          reset_index().
          rename(columns={'user_id' : 'total_events'}).
          sort_values(by='total_events', ascending=False))
events

In [ ]:
plt.figure(figsize=(15, 5))
ax = sns.barplot(x='total_events', y='event', data=events, alpha=0.5, color = 'blue')
ax.set_title('Частота событий в логах')
ax.set_xlabel('Количество') 
ax.set_ylabel('') 
plt.show()

**Что имеем:**

* MainScreenAppear (Главный экран) увидели 117328 раз;
* OffersScreenAppear (Каталог предложений) увидели 46333 раза;
* CartScreenAppear (Карточка товара) увидели 42303 раза;
* PaymentScreenSuccessful (Экран с подтверждением успешной оплаты) увидели 33918 раз;
* Tutorial (Урок) просмотрели 1005 раз.

Теперь посчитаем, сколько пользователей совершали каждое из этих событий, а так же долю пользователей, которые хоть раз совершали событие.

In [ ]:
final_data = (data.
          groupby('event').
          agg({'user_id': 'nunique'}).
          reset_index().
          rename(columns={'user_id' : 'total_users'}).
          sort_values(by='total_users', ascending=False))
final_data['percent'] = round(final_data['total_users'] / data['user_id'].nunique() * 100, 2)
final_data

**Таким образом:**

* Главную страницу увидели 7419 пользователей (98.5% от общего числа пользователей) - почти все
* Каталог предложений просмотрели 4593 пользователей (61% от общего числа)
* Карточку товара 3734 пользователя (49.6% от общего числа)
* Завершили оплату 3539 пользователей (47% от общего числа)
* Просмотрели урок 840 пользователей (11% от общего числа)


Все события выстраиваются в логичную цепочку действий, кроме просмотра урока (Tutorial). Посмотреть урок можно на любом этапе или вообще пропустить данное действие. Чаще всего, интерфейс является интуитивно понятным. Поэтому шаг Tutorial из дальнейшей цепочки уберем.

Тогда наша воронка событий выглядит следующим образом:

* Главный экран
* Каталог предложений
* Карточка товара
* Экран с подтверждением успешной оплаты

Теперь посчитаем, какая доля пользователей проходит на следующий шаг воронки (от числа пользователей на предыдущем). То есть для последовательности событий A → B → C → D посчитаем отношение числа пользователей с событием B к количеству пользователей с событием A, отношение числа пользователей с событием C к количеству пользователей с событием B и т.д. 

Визуализируем нашу воронку.

In [ ]:
final_data = final_data[final_data['event'] != 'Tutorial']

fig = go.Figure(go.Funnel(y = final_data['event'],
                          x = final_data['total_users'],
                          opacity = 0.6,
                          textposition = 'inside',
                          textinfo = 'value + percent previous'))
fig.update_layout(title_text='Воронка событий')
fig.show()

In [ ]:
final_data['percent_step'] = round(final_data['total_users'] / 
                                     final_data['total_users']
                                     .shift(periods=1, fill_value=final_data['total_users'][1]/1), 2)*100
display(final_data)

print('Посетили главную страницу:', final_data['percent_step'][1],'%')
print('Просмотрели Каталог в % от предыдущего шага:', final_data['percent_step'][2])
print('Просмотрели Карточку товара в % от предыдущего шага:', final_data['percent_step'][0])
print('Оплатили в % от предыдущего шага:', final_data['percent_step'][3])

**Вывод:** Мы видим, что наибольшее количество пользователей теряется после первого шага (38%). Это достаточно внушительный процент. 

Это может быть связано с тем, что: на главной странице показывается поп ап окно с информацией и пользователь просто закрывает приложение, или например для просмотра каталога необходимо внести свой адрес доставки, но не все готовы это делать не ознакомившись с каталогом и закрывают приложение. 

С остальными этапами воронки все очень неплохо: те кто попадает в каталог с большей вероятностью совершают покупку: 95% от числа тех, кто просмотрел карточку товара. Но в конечном счете, только 47% от первоначального количества пользователей успешно оплачивают товары из корзины.

## Анализ результатов А/А/В-эксперимента

Мы знаем, что у нас есть 2 контрольные группы для А/А-эксперимента (246 и 247), чтобы проверить корректность всех механизмов и расчётов, и одна тестовая группа (В, 248).

Вспомним сколько у нас участников в каждой группе.

In [ ]:
data_group

Как мы выявили ранее, самым популярным событием является: MainScreenAppear (Посещение главной страницы). Посмотрим распределние пользователей по этому событию в группах.

In [ ]:
# посмотрим сколько пользователей совершили событие: MainScreenAppear в каждой группе
data_group_event = (data.
                    groupby(['event', 'group_id']).
                    agg({'user_id': 'nunique'}).
                    reset_index().
                    rename(columns={'user_id' : 'total_users'}).
                    sort_values(by=['group_id','total_users'], ascending=False))

data_group_event = data_group_event[data_group_event['event'] != 'Tutorial']

data_group_event['ratio'] = round(data_group_event['total_users']/data_group_event['total_users'].sum()*100)
data_group_event_1 = data_group_event[data_group_event['event'] == 'MainScreenAppear']
data_group_event_1

Внешне, группы очень похожи. Проверим гипотезу о равенстве выборок, и для начала проверим находят ли статистические критерии разницу между выборками 246 и 247 (А/А-тест). Используем Z-критерий (статистический тест, позволяющий определить, различаются ли два средних значения генеральной совокупности, когда дисперсии известны и размер выборки велик). 

Всего у нас 4 вида событий, итого: 4 A/A теста и 12 А/В. Из-за множественного стравнения есть вероятность ложнопозитивного результата. Поэтому стоит использовать либо поправку Бонферрони, либо метод Шидака. Чтобы повысить мощность стат.теста (т.е. вероятность не ошибиться) используем метод Шидака. Для удобства, напишем функцию.

In [ ]:
def z_test(group_1, group_2, event, alpha, n):
    
    # критический уровень статистической значимости с учетом метода Шидака
    shidak_alpha = 1 - (1-alpha)**(1/n)
 
    # число пользователей в группе 1 и группе 2:
    n_users = np.array([group_1['user_id'].nunique(), 
                        group_2['user_id'].nunique()])

    # число пользователей, совершивших событие в группе 1 и группе 2
    success = np.array([group_1[group_1['event'] == event]['user_id'].nunique(), 
                        group_2[group_2['event'] == event]['user_id'].nunique()])

    # пропорции успехов в группах:
    p1 = success[0]/n_users[0]
    p2 = success[1]/n_users[1]
    
    # пропорция успехов в комбинированном датасете:
    p_combined = (success[0] + success[1]) / (n_users[0] + n_users[1])

    # разница пропорций в датасетах
    difference = p1 - p2 

    # считаем статистику в ст.отклонениях стандартного нормального распределения
    z_value = difference /  np.sqrt(p_combined * (1 - p_combined) * (1/n_users[0] + 1/n_users[1]))

    # задаем стандартное нормальное распределение (среднее 0, ст.отклонение 1)
    distr = st.norm(0, 1)  

    p_value = (1 - distr.cdf(abs(z_value))) * 2   #тест двусторонний, удваиваем результат
    
    print('Событие:', event)
    print('p-значение: ', p_value)

    if p_value < shidak_alpha:
        print('Отвергаем нулевую гипотезу: между долями есть разница')
    else:
        print('Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными')

Нам нужно будет сопоставить доли по каждому событию между:

* контрольными группами 246 и 247;
* каждой из контрольной группы по отдельности и экспериментальной (246-248 и 247-248);
* объединенной контрольной группой и экспериментальной (246+247 и 248).

Введем основную и альтернативные гипотезы для всех попарных сравнений:

* **Нулевая гипотеза:** доли уникальных посетителей, побывавших на этапе воронки, одинаковы
* **Альтернативная гипотеза:** доли уникальных посетителей, побывавших на этапе воронки, отличаются

In [ ]:
# проверим, есть ли статистически значимая разница между контрольными группами 246 и 247:

for event in data_group_event['event'].unique():
    z_test(data[data['group_id'] == 246], data[data['group_id'] == 247], event,.05, 16)
    print()

Цель А/А-теста — показать, можно ли доверять результатам эксперимента, который будет запущен в тех же условиях, но уже с разными вариантами шрифта. Между контрольными группами ни по одному событию нет статистически достоверного отличия при заданном уровне значимости, а значит, приступаем к A/B-тестированию.

In [ ]:
# проверим, есть ли статистически значимая разница между контрольной группой 246 и эксперементальной 248:

for event in data_group_event['event'].unique():
    z_test(data[data['group_id'] == 246], data[data['group_id'] == 248], event, .05, 16)
    print()

При заданном уровне значимости у нас нет оснований считать группы 246 и 248 разными.

In [ ]:
# проверим, есть ли статистически значимая разница между контрольной группой 247 и эксперементальной 248:

for event in data_group_event['event'].unique():
    z_test(data[data['group_id'] == 247], data[data['group_id'] == 248], event,.05, 16)
    print()

При заданном уровне значимости у нас так же нет оснований считать группы 247 и 248 разными.

In [ ]:
# проверим есть ли статистически значимая разница между объединённой контрольной и экпериментальной 248 группами:

for event in data_group_event['event'].unique():
    z_test(data[data['group_id'] != 248], data[data['group_id'] == 248], event,.05, 16)
    print()

Как и в прошлых тестах нам не удалось отвергнуть нулевую гипотезу, у нас нет оснований считать доли объединённой контрольной и экпериментальной 248 групп разными. Из этого делаем вывод, что **изменение шрифтов во всём приложении на поведение пользователей не повлияло.**

## Выводы

В результате исследования были проанализированы поведение покупателей на основании логов пользователей, а так же, результаты А/А/В-теста. После предобработки данных было рассмотрено поведение 7534 пользователя мобильного приложения.

Была сформирована воронка событий. Одно событие (Tutorial) было исключено из анализа ввиду необязательного прохождения и отсутствия влияния на остальные шаги.
"Спускаясь" по воронке мы выявили, что:

* Главную страницу увидели 7419 пользователей (100% от общего числа пользователей);
* Страницу товара просмотрели 4593 пользователей (62% от общего числа);
* Карточку просмотрели 3734 пользователя (52% от общего числа);
* Завершили оплату 3539 пользователей (48% от общего числа).

БОльшее количество пользователей приложение теряло после первого шага (более 38%), чуть менее 9% на следующем и около 2% при переходе на последний шаг. Т.е. примерно 48% пользователей воспользовались приложением и оплатили заказ.

Это может быть связано с тем, что: 
1. на главной странице показывается поп ап окно с информацией и пользователь просто закрывает приложение,
2. или для просмотра каталога необходимо внести свой адрес доставки, но не все готовы это делать не ознакомившись с каталогом и закрывают приложение.

Если что-то подобное есть в приложении, то мы бы рекомендовали провести дополнительные А/В тесты. 
Для эксперементальной группы:
* отключить поп ап
* ввод адреса предлагать на этапе заказа товаров.

Далее, был проанализирован результат А/А/В-эксперимента(изменение шрифта во всем приложении), для этого были ипользованы логи событий за период (с 01/08/2019 по 07/08/2019).

В эксперименте были учтено поведение пользователей, разделенных на три группы:

* 246-ая - 2484 пользователя;
* 247-ая - 2513 пользователя;
* 248-ая - 2537 пользователя.

Нам необходимо было сопоставить доли пользователей по каждому событию между:

* контрольными группами 246 и 247;
* каждой из контрольной группы по отдельности и экспериментальной (246-248 и 247-248);
* объединенной контрольной группой и экспериментальной (246+247 и 248).

Множесто А/В-тестов, проведённых по каждому из событий, не обнаружили статистически значимой разницы между группами. Значит изменение шрифтов во всём приложении на поведение пользователей не повлияло.
